In [1]:
import face_recognition
import cv2
import numpy as np
import os

 
    

In [2]:
import sqlite3


In [3]:
# Conexión a la base de datos SQLite
sqlite_conn = sqlite3.connect('C:/Users/rfune3/marcas.sqlite')
sqlite_cursor = sqlite_conn.cursor()


In [4]:
marcas_sql = '''
CREATE TABLE IF NOT EXISTS marcas (
    CODEMP integer,
    FECHA  DATETIME ,
    ES_ENTRADA BOOLEAN ,
    FECHA_HORA DATETIME ,
    TIPO  VARCHAR(50),
    INDICE VARCHAR(50) ,
    msrepl_tran_version TEXT ,
    USUARIO VARCHAR(20),
    hex_aprobada BOOLEAN,
    hex_rechazoID SMALLINT,
    computerName VARCHAR(50) ,
    fechaRegistro DATETIME
    
)'''

sqlite_cursor.execute(marcas_sql)
#sqlite_conn.commit()
#sqlite_conn.close()


In [5]:
import uuid

In [6]:
uid = uuid.uuid4()
uid_str = str(uid)
print(uid_str)

76e3b48b-d99e-440e-b857-08fc7581c0b9


In [18]:
sqlite_cursor.execute("INSERT INTO marcas (CODEMP, FECHA, ES_ENTRADA, FECHA_HORA, TIPO , INDICE, msrepl_tran_version , USUARIO , hex_aprobada ,hex_rechazoID, computerName,fechaRegistro   ) VALUES (?, ?, ?, ? , ?, ? , ?, ? , ?, ? , ?, ?)",
                      ("226", "2023-06-05 00:00:00.000",0,"2023-06-05 07:09:51.000", "Procesada",uid_str, uid_str, "rfune3", None, None, "CO1152798",  "2023-06-05 07:09:51.833"))

#sqlite_conn.commit()
#sqlite_conn.close()


NameError: name 'now' is not defined

In [8]:
consulta_sql = "SELECT * FROM marcas "
sqlite_cursor.execute(consulta_sql)
resultados = sqlite_cursor.fetchall()

for fila in resultados:
    print(fila)

    
#sqlite_conn.close()

(226, '2023-06-05 00:00:00.000', 0, '2023-06-05 07:09:51.000', 'Procesada', '76e3b48b-d99e-440e-b857-08fc7581c0b9', '76e3b48b-d99e-440e-b857-08fc7581c0b9', 'rfune3', None, None, 'CO1152798', '2023-06-05 07:09:51.833')


In [24]:

import datetime



2023-07-07 21:59:39.480542


In [41]:
my_datetime = datetime.datetime.now()
one_day = datetime.timedelta(minutes=-5)
new_datetime = my_datetime + one_day
print(new_datetime)
#time(FECHA_HORA) >=
 
sqlite_cursor.execute(" SELECT count(*)  as registro FROM marcas where CODEMP = '{}' and time(FECHA_HORA) < '{}'   ".format(226, '07:09:51.000') ) 
resultados = sqlite_cursor.fetchall()

for fila in resultados:
    print(fila)

2023-07-07 22:19:43.928292
(1,)


In [ ]:
import face_recognition
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
#from os import listdir


# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

dir_faces = 'C:/Users/rfune3/fotosexpediente'
known_face_encodings = []
known_face_names = []


for filename in os.listdir(dir_faces):
    if filename.endswith(".jpg") or filename.endswith(".JPG"):
        imagen = face_recognition.load_image_file(os.path.join('fotosexpediente/', filename))
        print("imagenes",filename)
        ubicaciones_rostros = face_recognition.face_locations(imagen)
        for ubicacion_rostro in ubicaciones_rostros:
            print("imagen",ubicacion_rostro)
            known_face_encodings.append(face_recognition.face_encodings(imagen)[0])
            known_face_names.append(filename)
            top, right, bottom, left = ubicacion_rostro
            #plt.imshow(imagen[top:bottom, left:right])
            #plt.axis("off")
            #plt.show()
            
#contenido = os.listdir(dir_faces) 
#Cargar la imagen de ejemplo con nuestro rostro:
#imagen_personal = face_recognition.load_image_file(contenido)

#Extraer los 'encodings' que caracterizan nuestro rostro:
#personal_encodings = face_recognition.face_encodings(imagen_personal)[0]



# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)




# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("226.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("454782.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
#known_face_encodings = [
   # obama_face_encoding,
  #  biden_face_encoding,
 #   ubicaciones_rostros
  #  personal_encodings
#]
#known_face_names = [
#    "Ritchie Funes",
#    "osorto",
#    "jorge peralta"
#]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                sqlite_cursor.execute("INSERT INTO marcas (CODEMP, FECHA, ES_ENTRADA, FECHA_HORA, TIPO , INDICE, msrepl_tran_version , USUARIO , hex_aprobada ,hex_rechazoID, computerName,fechaRegistro   ) VALUES (?, ?, ?, ? , ?, ? , ?, ? , ?, ? , ?, ?)",
                      ("226", "2023-06-05 00:00:00.000",0,"2023-06-05 07:09:51.000", "Procesada",uid_str, uid_str, "rfune3", None, None, "CO1152798",  "2023-06-05 07:09:51.833"))

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

imagenes 103.JPG
imagen (1125, 1763, 2275, 613)
imagenes 110.JPG
imagen (30, 146, 105, 71)
imagenes 110182.jpg
imagen (44, 187, 152, 79)
imagenes 110226.jpg
imagen (46, 185, 136, 96)
imagenes 110230.jpg
imagen (118, 283, 304, 97)
imagenes 113366.JPG
imagen (116, 270, 270, 115)
imagenes 123.JPG
imagen (39, 156, 101, 93)
imagenes 154.jpg
imagen (290, 632, 675, 247)
imagenes 155378.JPG
imagen (46, 145, 136, 56)
imagenes 161235.jpg.jpg
imagen (38, 118, 90, 66)
imagenes 163247.JPG
imagen (55, 129, 130, 55)
imagenes 163702.JPG
imagen (56, 163, 163, 55)
imagenes 164992.jpg
imagen (46, 156, 108, 93)
imagenes 226.jpg
imagen (53, 182, 182, 53)
imagenes 301027.jpg
imagen (56, 151, 163, 44)
imagenes 351686.jpg
imagen (47, 137, 121, 63)
imagenes 450175.jpg
imagen (46, 156, 108, 93)
imagenes 453562.jpg
imagen (56, 163, 163, 55)
imagenes 454782.JPG
imagen (55, 113, 130, 38)
imagenes 455218.jpg
imagen (96, 210, 225, 81)
imagenes 455322.jpg
imagen (59, 87, 122, 24)
imagenes 49608.JPG
imagen (47, 146, 1

In [ ]:
consulta_sql = "SELECT * FROM marcas "
sqlite_cursor.execute(consulta_sql)
resultados = sqlite_cursor.fetchall()

for fila in resultados:
    print(fila)

In [ ]:
import face_recognition
import cv2
import numpy as np
import os
#from os import listdir


# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

dir_faces = 'http://localhost:8889/tree/fotosexpediente'


            
#contenido = os.listdir(dir_faces) 
#Cargar la imagen de ejemplo con nuestro rostro:
#imagen_personal = face_recognition.load_image_file(contenido)

#Extraer los 'encodings' que caracterizan nuestro rostro:
#personal_encodings = face_recognition.face_encodings(imagen_personal)[0]



# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)




# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("226.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("454782.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
  #  personal_encodings
]
known_face_names = [
    "Ritchie Funes",
    ""
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
consulta_sql = "SELECT * FROM marcas "
sqlite_cursor.execute(consulta_sql)
resultados = sqlite_cursor.fetchall()

for fila in resultados:
    print(fila)

1.2.3


AttributeError: module 'face_recognition' has no attribute 'face_distances'